In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# creating df of data
csv_file = 'gym_members_exercise_tracking.csv'
data = pd.read_csv(csv_file)

features = data.drop(['Calories_Burned','Water_Intake (liters)', 'Workout_Frequency (days/week)'], axis=1)
target = data['Calories_Burned']

In [5]:
# encoding non numberical values
encoded_features = pd.get_dummies(features, columns=['Workout_Type', 'Gender'])

# creating train and test dataframes
X_train, X_test, y_train, y_test = train_test_split(encoded_features, target, test_size=.2, random_state=42)

# scaling numerical values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(X_train_scaled.shape[1]),
    
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(1)
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  restore_best_weights=True
                                                  )

optimizer = tf.keras.optimizers.Adam(learning_rate=.002)

model.compile(optimizer=optimizer,
                        loss='mse',
                        metrics=['mae']
                        )

history = model.fit(X_train_scaled,
                    y_train,
                    validation_split=.2,
                    validation_steps=10,
                    shuffle=True,
                    epochs=500,
                    callbacks=[early_stopping],
                    verbose=2
                    )

Epoch 1/500
20/20 - 2s - loss: 879196.0000 - mae: 898.3040 - val_loss: 920768.9375 - val_mae: 924.4631 - 2s/epoch - 123ms/step
Epoch 2/500
20/20 - 0s - loss: 875281.6875 - mae: 896.9061 - val_loss: 916482.4375 - val_mae: 922.8589 - 132ms/epoch - 7ms/step
Epoch 3/500
20/20 - 0s - loss: 871628.8750 - mae: 895.3559 - val_loss: 911159.0000 - val_mae: 920.5588 - 137ms/epoch - 7ms/step
Epoch 4/500
20/20 - 0s - loss: 867525.0625 - mae: 893.4630 - val_loss: 905598.5625 - val_mae: 918.0352 - 132ms/epoch - 7ms/step
Epoch 5/500
20/20 - 0s - loss: 862977.3750 - mae: 891.3459 - val_loss: 898820.4375 - val_mae: 915.0523 - 135ms/epoch - 7ms/step
Epoch 6/500
20/20 - 0s - loss: 857691.9375 - mae: 888.8725 - val_loss: 894587.8750 - val_mae: 912.9965 - 125ms/epoch - 6ms/step
Epoch 7/500
20/20 - 0s - loss: 851887.2500 - mae: 886.0637 - val_loss: 886105.6250 - val_mae: 909.2794 - 128ms/epoch - 6ms/step
Epoch 8/500
20/20 - 0s - loss: 845261.3750 - mae: 883.0305 - val_loss: 875769.9375 - val_mae: 904.3709 - 

In [7]:
results = model.evaluate(X_test_scaled, y_test)
print("Test loss:", results[0])
print("Test mae:", results[1])

7/7 [==============================] - 0s 2ms/step - loss: 1217.6079 - mae: 26.9536
Test loss: 1217.60791015625
Test mae: 26.95355987548828


In [8]:
# preparing prediction data
prediction_data = pd.read_csv('test_data.csv')
prediction_data = prediction_data.drop(['Water_Intake (liters)', 'Workout_Frequency (days/week)'], axis=1)

prediction_data = pd.get_dummies(prediction_data)

# preprocessing data
scaled_prediction_data = scaler.fit_transform(prediction_data)

scaled_predictions = model.predict(scaled_prediction_data)

print(scaled_predictions)


1/1 [==============================] - 0s 99ms/step
[[1326.4625 ]
 [ 978.2983 ]
 [ 723.84674]
 [ 523.0234 ]
 [ 515.11414]
 [1156.5039 ]
 [1293.2539 ]
 [ 897.7216 ]
 [ 706.69104]
 [ 756.8455 ]]


In [286]:
model.save('calorie_predictions.keras')